In [1]:
 class Node(object):
    def __init__(self, parent=None, left=None, right=None, weight=0, symbol=''):
        super(Node, self).__init__()
        self.parent = parent
        self.left = left
        self.right = right
        self.weight = weight
        self.symbol = symbol
   
    def parent(self):
        return self.parent
   
    def parent(self, parent):
        self.parent = parent
    
    def left(self):
        return self.left
   
    def left(self, left):
        self.left = left
    
    def right(self):
        return self.right
    
    def right(self, right):
        self.right = right
    
    def weight(self):
        return self.weight
    
    def weight(self, weight):
        self.weight = weight
   
    def symbol(self):
        return self.symbol
    
    def symbol(self, symbol):
        self.symbol = symbol

In [2]:
class FGK(object):
    def __init__(self):
        super(FGK, self).__init__()
        self.NYT = Node(symbol="NYT")
        self.root = self.NYT
        self.nodes = []
        self.seen = [None] * 256

    def get_code(self, s, node, code=''):
        if node.left is None and node.right is None:
            return code if node.symbol == s else ''
        else:
            temp = ''
            if node.left is not None:
                temp = self.get_code(s, node.left, code+'0')
            if not temp and node.right is not None:
                temp = self.get_code(s, node.right, code+'1')
            return temp

    def find_largest_node(self, weight):
        for n in reversed(self.nodes):
            if n.weight == weight:
                return n

    def swap_node(self, n1, n2):
        i1, i2 = self.nodes.index(n1), self.nodes.index(n2)
        self.nodes[i1], self.nodes[i2] = self.nodes[i2], self.nodes[i1]
        tmp_parent = n1.parent
        n1.parent = n2.parent
        n2.parent = tmp_parent
        if n1.parent.left is n2:
            n1.parent.left = n1
        else:
            n1.parent.right = n1
        if n2.parent.left is n1:
            n2.parent.left = n2
        else:
            n2.parent.right = n2

    def insert(self, s):
        node = self.seen[ord(s)]

        if node is None:
            spawn = Node(symbol=s, weight=1)
            internal = Node(symbol='', weight=1, parent=self.NYT.parent, left=self.NYT, right=spawn)
            spawn.parent = internal
            self.NYT.parent = internal

            if internal.parent is not None:
                internal.parent.left = internal
            else:
                self.root = internal
            self.nodes.insert(0, internal)
            self.nodes.insert(0, spawn)
            self.seen[ord(s)] = spawn
            node = internal.parent

        while node is not None:
            largest = self.find_largest_node(node.weight)
            if (node is not largest and node is not largest.parent and largest is not node.parent):
                self.swap_node(node, largest)

            node.weight = node.weight + 1
            node = node.parent

    def encode(self, text):
        result = ''
        for s in text:
            if self.seen[ord(s)]:
                result += self.get_code(s, self.root)
            else:
                result += self.get_code('NYT', self.root)
                result += bin(ord(s))[2:].zfill(8)
            self.insert(s)
        return result

    def get_symbol_by_ascii(self, bin_str):
        return chr(int(bin_str,2))

    def decode(self, text):
        result = ''
        symbol = self.get_symbol_by_ascii(text[:8])
        result += symbol
        self.insert(symbol)
        node = self.root
        i = 8
        while i < len(text):
            node = node.left if text[i] == '0' else node.right
            symbol = node.symbol
            if symbol:
                if symbol == 'NYT':
                    symbol = self.get_symbol_by_ascii(text[i+1:i+9])
                    i += 8
                result += symbol
                self.insert(symbol)
                node = self.root
            i += 1
        return result




In [3]:
text = None
result = None

with open("sample.txt") as f:
    text = f.read()

print("Original text file:\n\n", text)

Original text file:

 We're no strangers to love
You know the rules and so do I
A full commitment's what I'm thinking of
You wouldn't get this from any other guy
I just wanna tell you how I'm feeling
Gotta make you understand
Never gonna give you up, never gonna let you down
Never gonna run around and desert you
Never gonna make you cry, never gonna say goodbye
Never gonna tell a lie and hurt you
We've known each other for so long
Your heart's been aching but you're too shy to say it
Inside we both know what's been going on
We know the game and we're gonna play it
And if you ask me how I'm feeling
Don't tell me you're too blind to see
Never gonna give you up, never gonna let you down
Never gonna run around and desert you
Never gonna make you cry, never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up, never gonna let you down
Never gonna run around and desert you
Never gonna make you cry, never gonna say goodbye
Never gonna tell a lie and hurt you
We've kno

In [4]:
result = FGK().encode(text)
print("Encoded text file:\n\n", result)

Encoded text file:

 0101011100110010100001001111000111001011000001000001000011011100100011011110110000011100111100001110100110010100011000011010011000110011111010010111000111011010100100011011001110011100011101101001010000000101000110001011001010011000011101011100001000110101111000100010000111011111001001110100011010000101101100001010010001101011100001010100000000110010011001110100010100010100010100000100100111101010000000100000100010100001100110110011100011011001110110001100011010111110000110110111010101001010001101001110110010011011011000111001010011110011000010100010010101110111010000011000111101011011000001011000000010110101010000000111111111110110111100001100011110110011111011100110110000011011010010110010001111100001000100111111000110001101110011111011111011000110010111011001010100100000111100111100111010111010001011111100100101110000010101000011101111010101000110101011010110101100111110100011001001100101111111110010001001010011111010100111011111111100100000100011110111101100011

In [5]:
result = FGK().decode(result)
print ("Decoded text file:\n\n", result)

Decoded text file:

 We're no strangers to love
You know the rules and so do I
A full commitment's what I'm thinking of
You wouldn't get this from any other guy
I just wanna tell you how I'm feeling
Gotta make you understand
Never gonna give you up, never gonna let you down
Never gonna run around and desert you
Never gonna make you cry, never gonna say goodbye
Never gonna tell a lie and hurt you
We've known each other for so long
Your heart's been aching but you're too shy to say it
Inside we both know what's been going on
We know the game and we're gonna play it
And if you ask me how I'm feeling
Don't tell me you're too blind to see
Never gonna give you up, never gonna let you down
Never gonna run around and desert you
Never gonna make you cry, never gonna say goodbye
Never gonna tell a lie and hurt you
Never gonna give you up, never gonna let you down
Never gonna run around and desert you
Never gonna make you cry, never gonna say goodbye
Never gonna tell a lie and hurt you
We've know

In [6]:
text == result

True